In [3]:
import os
import sys

import xarray as xr
import numpy as np
import cv2
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import umap

# Get the current working directory
CURRENT_DIR = os.getcwd()

# Set the root directory to the parent of the current directory
ROOT = os.path.abspath(os.path.join(CURRENT_DIR, os.pardir))
sys.path.insert(0, ROOT)

from code_.tools.loading import load_nsd_images
from code_.encoding_score.benchmarks.nsd import filter_activations, load_nsd_data
from code_.model_activations.loading import load_full_identifier

from config import RESULTS, FIGURES, CACHE



# tSNE

In [4]:

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import umap
from sklearn.manifold import TSNE

# Ensure required packages are installed: pip install umap-learn opencv-python matplotlib scikit-learn

def scale_to_01_range(x):
    """Scale values between 0 and 1."""
    value_range = np.max(x) - np.min(x)
    return (x - np.min(x)) / value_range if value_range != 0 else x

def compute_plot_coordinates(image, x, y, image_centers_area_size, offset):
    """Compute image positions on the plot."""
    image_height, image_width, _ = image.shape
    center_x = int(image_centers_area_size * x) + offset
    center_y = int(image_centers_area_size * (1 - y)) + offset  # Mirror y-axis

    tl_x = center_x - int(image_width / 2)
    tl_y = center_y - int(image_height / 2)
    br_x = tl_x + image_width
    br_y = tl_y + image_height

    return tl_x, tl_y, br_x, br_y

def scale_image(image, max_image_size):
    """Resize images while maintaining aspect ratio."""
    image_height, image_width, _ = image.shape
    scale = max(1, image_width / max_image_size, image_height / max_image_size)
    image_width = int(image_width / scale)
    image_height = int(image_height / scale)

    return cv2.resize(image, (image_width, image_height))

def get_embeddings(data, method="UMAP"):
    """
    Perform dimensionality reduction (UMAP or t-SNE) and visualize images.
    
    Parameters:
    - data: (N, D) numpy array of feature vectors.
    - images: List of image paths corresponding to each data point.
    - name: Output filename.
    - method: Choose between "UMAP" and "tSNE".
    - dpi: Resolution of the output plot.
    - plot_size: Size of the final visualization.
    - max_image_size: Max image size on the plot.
    """

    # Perform dimensionality reduction
    if method.lower() == "umap":
        reducer = umap.UMAP(n_components=2, random_state=42)
        embedding = reducer.fit_transform(data)
    elif method.lower() == "tsne":
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
        embedding = reducer.fit_transform(data)
    else:
        raise ValueError("Invalid method! Choose 'UMAP' or 'tSNE'.")

    return embedding
    

def visualize_embeddings(embedding, images, save_path,  method, dpi=300, plot_size=1000, max_image_size=50):
    # Normalize the reduced output to [0,1] range
    tx = scale_to_01_range(embedding[:, 0])
    ty = scale_to_01_range(embedding[:, 1])

    # Set up the visualization plot
    offset = max_image_size // 2
    image_centers_area_size = plot_size - 2 * offset
    plot_canvas = 255 * np.ones((plot_size, plot_size, 3), np.uint8)

    # Place images on the scatter plot
    for i in range(len(images)):
        x, y = tx[i], ty[i]
        try:
            image = cv2.imread(images[i])
            image = scale_image(image, max_image_size)
            tl_x, tl_y, br_x, br_y = compute_plot_coordinates(image, x, y, image_centers_area_size, offset)
            plot_canvas[tl_y:br_y, tl_x:br_x, :] = image  # Place image on plot
        except AttributeError:
            pass  # Ignore images that fail to load

    # Plot and save figure
    fig = plt.figure(figsize=(80, 50), dpi=300)
    plt.imshow(plot_canvas[:, :, ::-1])
    plt.axis("off")
    plt.title(f"{method} Visualization", fontsize=50)
    plt.savefig(save_path, dpi=dpi)
    plt.show()

In [5]:
# load model activations
CACHE = '/data/atlas/.repo_cache_10'
identifier = load_full_identifier(model_name='expansion', features=3, layers=5, dataset='naturalscenes')
features = xr.open_dataarray(os.path.join(CACHE,'activations',identifier),engine='netcdf4')

# load images seen by subject 1
subject = 1
IDS = load_nsd_data(mode='unshared', subject=subject, region='early visual stream')[0]
images  = [image for image in load_nsd_images() if os.path.basename(image).strip('.png') in IDS]

# filter activations for images seen by subject 1
features = filter_activations(features, IDS)

# tsne_ints = TSNE(n_components=2).fit_transform(features)
# tx = tsne_ints[:, 0]
# ty = tsne_ints[:, 1]
# tx = scale_to_01_range(tx)
# ty = scale_to_01_range(ty)
# dpi  = 800
# # visualize 
# visualize_tsne_images(tx, ty, images, 
#                       dpi = dpi, 
#                       plot_size=1000, 
#                       max_image_size=10)

In [4]:
# plot umap or tsne
method = 'tsne'
save_path = os.path.join(FIGURES, f'{method.lower()}- subject={subject}.png')
embedding = get_embeddings(features, method)

In [2]:
visualize_embeddings(embedding, images, save_path,  method, max_image_size=10)

NameError: name 'visualize_embeddings' is not defined